In [2]:
import sqlite3
import pandas as pd
import numpy as np

# DataDict

In [2]:
df = pd.read_excel('DataDict_FAOFish.xlsx')
df = df[df['Notes'] == 'KS']
df

,Variable,Table,Group,SubGroup,Series,CoVaTrA,Cohort,Definition,Extended Source Defn,Units,...,Proprietary,Name in Source,UsedInPreprocessor,UsedInPreprocessorFileName,CompareOtherForecast,Code in Source,Decimal Places,Country Concordance,Formula,DisplayNotes
11,AgFishCatchProdAqAnimalsFSJ,SeriesAgFishCatchProdAqAnimalsFSJ,Agriculture,Fish,Yes,No,No,Total Aquatic Animals Catch Production (tonnes...,NaN,Tonnes,...,0,Aquatic Animals NEI Global Capture Production;...,1,AGRI,0,Aquatic Animals NEI,4,FAO Fishstat,NaN,NaN
12,AgFishCatchProdAqPlantsFSJ,SeriesAgFishCatchProdAqPlantsFSJ,Agriculture,Fish,Yes,No,No,Total Aquatic Plants Catch Production (tonnes)...,NaN,Tonnes,...,0,Aquatic Plants Global Capture Production; All ...,1,AGRI,0,Aquatic Plants,4,FAO Fishstat,NaN,NaN
13,AgFishCatchProdCephalopodsFSJ,SeriesAgFishCatchProdCephalopodsFSJ,Agriculture,Fish,Yes,No,No,Total Cephalopods Capture Production from Fish...,NaN,Tonnes,...,0,Cephalapods Global Capture Production; All fis...,1,AGRI,0,Cephalopods,4,FAO Fishstat,NaN,NaN
14,AgFishCatchProdCrustaceansFSJ,SeriesAgFishCatchProdCrustaceansFSJ,Agriculture,Fish,Yes,No,No,Total Crustaceans Capture Production from Fish...,NaN,Tonnes,...,0,Crustaceans Global Capture Production; All fis...,1,AGRI,0,Crustaceans,4,FAO Fishstat,NaN,NaN
15,AgFishCatchProdDemersalFSJ,SeriesAgFishCatchProdDemersalFSJ,Agriculture,Fish,Yes,No,No,Total Demersal Capture Production from FishstatJ,NaN,Tonnes,...,0,Demersal Marine Fish Global Capture Production...,1,AGRI,0,Demersal Marine Fish,4,FAO Fishstat,NaN,NaN
16,AgFishCatchProdFreshwaterFSJ,SeriesAgFishCatchProdFreshwaterFSJ,Agriculture,Fish,Yes,No,No,Total Freshwater Capture Production from Fishs...,NaN,Tonnes,...,0,Freshwater and Diadromous Fish Global Capture ...,1,AGRI,0,Freshwater and Diadromous Fish,4,FAO Fishstat,NaN,NaN
17,AgFishCatchProdMarineFSJ,SeriesAgFishCatchProdMarineFSJ,Agriculture,Fish,Yes,No,No,Total Marine Capture Production from FishstatJ,NaN,Tonnes,...,0,Marine Fish NEI Global Capture Production; All...,1,AGRI,0,Marine Fish NEI,4,FAO Fishstat,NaN,NaN
18,AgFishCatchProdMolluscsFSJ,SeriesAgFishCatchProdMolluscsFSJ,Agriculture,Fish,Yes,No,No,Total Molluscs Capture Production from FishstatJ,NaN,Tonnes,...,0,Molluscs excl. cephalopods Global Capture Prod...,1,AGRI,0,Molluscs excl. Cephalopods,4,FAO Fishstat,NaN,NaN
19,AgFishCatchProdPelagicFSJ,SeriesAgFishCatchProdPelagicFSJ,Agriculture,Fish,Yes,No,No,Total Pelagic Capture Production (tonnes) from...,NaN,Tonnes,...,0,Pelagic marine fish Global Capture Production;...,1,AGRI,0,Pelagic Marine Fish,4,FAO Fishstat,NaN,NaN
20,AgFishExportValue%GDPFAOTrade,SeriesAgFishExportValue%GDPFAOTrade,"Agriculture, Trade",Trade,Yes,No,No,Export value of fish as a percent of GDP from ...,Export value of fish ($1000 US) from Fishstat...,Percent,...,0,Export Value,1,AGRI,0,NaN,4,FAO Countries,NaN,NaN


# Country

In [7]:
country = pd.read_excel('Country.xlsx', sheet_name='Production')
country_new = pd.read_excel('IFsCountry.xlsx')

col = ['Country (Name)']
for y in range(1950,2022):
    col.append(f'[{y}]')

col_new = ['Country (Name)']
for y in range(1950,2022):
    col_new.append(y)

In [4]:
dt1 = pd.read_csv('CatchProduction-1.csv', encoding='windows-1252')
len(dt1['Country (Name)'].unique())
dt2 = pd.read_csv('ExportImportValue-1.csv', encoding='windows-1252')
len(dt2['Reporting country (Name)'].unique())
for i in dt2['Reporting country (Name)'].unique():
    if i not in dt1['Country (Name)'].unique():
        print(i)

Ethiopia PDR
Totals - Value (USD 1000)
FAO. 2023. Fishery and Aquaculture Statistics. Global Aquatic Trade - All partners aggregated 1976-2021 (FishStatJ). In: FAO Fisheries and Aquaculture Division [online]. Rome. Updated 2023. 
www.fao.org/fishery/en/statistics/software/fishstatj


# CaptureProduction

In [5]:
cp = df.head(9)
for i in range(len(cp)):
    v = cp.iloc[i,0]
    s = cp.iloc[i,24]
    print(v, s)

AgFishCatchProdAqAnimalsFSJ Aquatic Animals NEI
AgFishCatchProdAqPlantsFSJ Aquatic Plants
AgFishCatchProdCephalopodsFSJ Cephalopods
AgFishCatchProdCrustaceansFSJ Crustaceans
AgFishCatchProdDemersalFSJ Demersal Marine Fish
AgFishCatchProdFreshwaterFSJ Freshwater and Diadromous Fish
AgFishCatchProdMarineFSJ Marine Fish NEI
AgFishCatchProdMolluscsFSJ Molluscs excl. Cephalopods
AgFishCatchProdPelagicFSJ Pelagic Marine Fish


## Loop to generate the files

In [8]:
for i in range(len(cp)):
    v = cp.iloc[i,0]
    s = cp.iloc[i,24]
    print(v, s)
    dt = pd.read_csv('CatchProduction-1.csv', encoding='windows-1252')
    dt = dt[dt['ASFIS species (Name)'] == s]
    dt = dt[col]
    dt.columns = col_new
    dt = dt.set_index('Country (Name)')
    dt = dt.replace("...","")
    dt = pd.DataFrame(dt.stack(dropna = False), 
                    columns = ['val']).reset_index().rename(columns={'level_1':'year'})

    dt = country.merge(dt, left_on = 'FAOFISH_Country', right_on = 'Country (Name)', how = 'left')
    dt = dt.drop(columns=['FAOFISH_Country', 'Country (Name)'])
    dt = dt.groupby(['Country', 'FIPS_CODE', 'year']).sum().reset_index()

    dt.year = dt.year.astype("Int64")

    dt1 = dt[['Country','year','val']]
    first_last = dt1.sort_values(['Country','year']).groupby(dt1['Country'])['val'] \
            .agg(['first', 'last']).reset_index()
    first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})

    dt = dt.pivot_table(index = ['Country','FIPS_CODE'],
                        columns='year',
                        values = 'val').reset_index()

    dt = dt.merge(first_last,on=['Country'],how='left')
    dt = country_new.merge(dt, on = ['Country','FIPS_CODE'], how = 'left')
    print(dt.shape)

    # first_last

    # dt.to_csv(f'IFs Import/{v}.csv', index = False)

AgFishCatchProdAqAnimalsFSJ Aquatic Animals NEI
(188, 76)
AgFishCatchProdAqPlantsFSJ Aquatic Plants
(188, 76)
AgFishCatchProdCephalopodsFSJ Cephalopods
(188, 76)
AgFishCatchProdCrustaceansFSJ Crustaceans
(188, 76)
AgFishCatchProdDemersalFSJ Demersal Marine Fish
(188, 76)
AgFishCatchProdFreshwaterFSJ Freshwater and Diadromous Fish
(188, 76)
AgFishCatchProdMarineFSJ Marine Fish NEI
(188, 76)
AgFishCatchProdMolluscsFSJ Molluscs excl. Cephalopods
(188, 76)
AgFishCatchProdPelagicFSJ Pelagic Marine Fish
(188, 76)


# Export/Import

## GDP

In [9]:
gdp = pd.read_csv('GDPCurrentUSDollar.csv') # download GDP Current $ from WDI
col = ['Country Name']
for y in range(1960,2022):
    col.append(f'{y} [YR{y}]')

col_new = ['Country Name']
for y in range(1960,2022):
    col_new.append(y)

gdp = gdp[col]
gdp.columns = col_new
gdp = gdp.replace("..","")

country_wdi = pd.read_excel('Country.xlsx', sheet_name='WDI')
gdp = country_wdi.merge(gdp, left_on='Country_WDI', right_on = 'Country Name', how = 'left' )
gdp = gdp.drop(columns=['Country_WDI', 'Country Name'])
gdp = gdp.set_index(['Country','FIPS_CODE'])
gdp = pd.DataFrame(gdp.stack(dropna = False), 
                  columns = ['val']).reset_index().rename(columns={'level_2':'year'})

gdp.val = pd.to_numeric(gdp.val) #.astype(float)
gdp

,Country,FIPS_CODE,year,val
0,Afghanistan,AFG,1960,5.377778e+08
1,Afghanistan,AFG,1961,5.488889e+08
2,Afghanistan,AFG,1962,5.466667e+08
3,Afghanistan,AFG,1963,7.511112e+08
4,Afghanistan,AFG,1964,8.000000e+08
...,...,...,...,...
11651,Zimbabwe,ZWE,2017,1.758489e+10
11652,Zimbabwe,ZWE,2018,3.415607e+10
11653,Zimbabwe,ZWE,2019,2.183223e+10
11654,Zimbabwe,ZWE,2020,2.150970e+10


## AgFishExportValue%GDPFAOTrade

### AgFishExportValue

In [10]:
country = pd.read_excel('Country.xlsx', sheet_name='ExportImport')
# country_new = pd.read_excel(r'C:\Users\panda\OneDrive - University of Denver\Pardee 2024 Winter\IFsCountry.xlsx')

col = ['Reporting country (Name)']
for y in range(1976,2022):
    col.append(f'[{y}]')

col_new = ['Reporting country (Name)']
for y in range(1976,2022):
    col_new.append(y)
    
dt = pd.read_csv('ExportImportValue-1.csv', encoding='windows-1252')
dt = dt[dt['Trade flow (Name)'] == 'Exports']
dt = dt[col]
dt.columns = col_new
dt = dt.set_index('Reporting country (Name)')
dt = dt.replace("...","")
dt = pd.DataFrame(dt.stack(dropna = False), 
                  columns = ['val']).reset_index().rename(columns={'level_1':'year'})

dt = country.merge(dt, left_on = 'FAOFISH_Country', right_on = 'Reporting country (Name)', how = 'left')
dt = dt.drop(columns=['FAOFISH_Country', 'Reporting country (Name)'])
dt = dt.groupby(['Country', 'FIPS_CODE', 'year']).sum().reset_index()

dt.year = dt.year.astype("Int64")

dt.val = dt.val*1000
dt

,Country,FIPS_CODE,year,val
0,Afghanistan,AFG,1976,0.0
1,Afghanistan,AFG,1977,0.0
2,Afghanistan,AFG,1978,0.0
3,Afghanistan,AFG,1979,0.0
4,Afghanistan,AFG,1980,0.0
...,...,...,...,...
8505,Zimbabwe,ZWE,2017,8067000.0
8506,Zimbabwe,ZWE,2018,6533000.0
8507,Zimbabwe,ZWE,2019,6951640.0
8508,Zimbabwe,ZWE,2020,4160180.0


In [11]:
final = dt.merge(gdp, on = ['Country', 'FIPS_CODE', 'year'], how = 'left')
final['val'] = round(final['val_x']/final['val_y']*100,4)
dt1 = final[['Country','year','val']]
first_last = dt1.sort_values(['Country','year']).groupby(dt1['Country'])['val'] \
        .agg(['first', 'last']).reset_index()
first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})

final = final.pivot_table(index = ['Country','FIPS_CODE'],
                    columns='year',
                    values = 'val').reset_index()

final = final.merge(first_last, on=['Country'],how='left')
final = country_new.merge(final, on = ['Country','FIPS_CODE'], how = 'left')

# final.to_csv('IFs Import/AgFishExportValue%GDPFAOTrade.csv', index = False)
final

,Country,FIPS_CODE,1976,1977,1978,1979,1980,1981,1982,1983,...,2014,2015,2016,2017,2018,2019,2020,2021,Earliest,MostRecent
0,Afghanistan,AFG,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,NaN,NaN,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0001,0.0001,0.0000,0.0000,0.0000
1,Albania,ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.3486,0.4212,0.5327,0.7223,0.7105,0.7238,0.7838,0.8513,0.0256,0.8513
2,Algeria,DZA,0.0267,0.0079,0.0014,0.0009,0.0009,0.0008,0.0001,0.0003,...,0.0033,0.0038,0.0042,0.0056,0.0070,0.0098,0.0130,0.0126,0.0267,0.0126
3,Angola,AGO,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000,...,0.0529,0.0819,0.1871,0.0944,0.1159,0.0368,0.1530,0.0689,0.0000,0.0689
4,Argentina,ARG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1536,...,0.3015,0.2466,0.3051,0.3074,0.3968,0.3982,0.4275,0.3882,0.1536,0.3882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,"Venezuela, Bolivarian Republic",VEN,0.0824,0.0386,0.0344,0.0196,0.0083,0.0187,0.0356,0.0821,...,0.0039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0824,0.0039
184,Viet Nam,VNM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.4468,2.8313,2.8566,3.0518,2.8737,2.6003,2.4565,2.4820,0.5249,2.4820
185,Yemen,YEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.4903,0.3160,0.3918,0.4082,0.5731,NaN,NaN,NaN,0.1275,0.5731
186,Zambia,ZMB,0.0004,0.0004,0.0000,0.0000,0.0000,0.0002,0.0000,0.0000,...,0.0027,0.0025,0.0026,0.0039,0.0038,0.0052,0.0203,0.0705,0.0004,0.0705


## AgFishImportValue%GDPFAOTrade

### AgFishImportValue

In [12]:
country = pd.read_excel('Country.xlsx', sheet_name='ExportImport')
# country_new = pd.read_excel(r'C:\Users\panda\OneDrive - University of Denver\Pardee 2024 Winter\IFsCountry.xlsx')

col = ['Reporting country (Name)']
for y in range(1976,2022):
    col.append(f'[{y}]')

col_new = ['Reporting country (Name)']
for y in range(1976,2022):
    col_new.append(y)
    
dt = pd.read_csv('ExportImportValue-1.csv', encoding='windows-1252')
dt = dt[dt['Trade flow (Name)'] == 'Imports']
dt = dt[col]
dt.columns = col_new
dt = dt.set_index('Reporting country (Name)')
dt = dt.replace("...","")
dt = pd.DataFrame(dt.stack(dropna = False), 
                  columns = ['val']).reset_index().rename(columns={'level_1':'year'})

dt = country.merge(dt, left_on = 'FAOFISH_Country', right_on = 'Reporting country (Name)', how = 'left')
dt = dt.drop(columns=['FAOFISH_Country', 'Reporting country (Name)'])
dt = dt.groupby(['Country', 'FIPS_CODE', 'year']).sum().reset_index()

dt.year = dt.year.astype("Int64")

dt.val = dt.val*1000
dt

,Country,FIPS_CODE,year,val
0,Afghanistan,AFG,1976,0.0
1,Afghanistan,AFG,1977,0.0
2,Afghanistan,AFG,1978,0.0
3,Afghanistan,AFG,1979,0.0
4,Afghanistan,AFG,1980,0.0
...,...,...,...,...
8505,Zimbabwe,ZWE,2017,21034000.0
8506,Zimbabwe,ZWE,2018,16232000.0
8507,Zimbabwe,ZWE,2019,6011790.0
8508,Zimbabwe,ZWE,2020,7977250.0


In [13]:
final = dt.merge(gdp, on = ['Country', 'FIPS_CODE', 'year'], how = 'left')
final['val'] = round(final['val_x']/final['val_y']*100,4)
dt1 = final[['Country','year','val']]
first_last = dt1.sort_values(['Country','year']).groupby(dt1['Country'])['val'] \
        .agg(['first', 'last']).reset_index()
first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})

final = final.pivot_table(index = ['Country','FIPS_CODE'],
                    columns='year',
                    values = 'val').reset_index()

final = final.merge(first_last, on=['Country'],how='left')
final = country_new.merge(final, on = ['Country','FIPS_CODE'], how = 'left')

# final.to_csv('IFs Import/AgFishImportValue%GDPFAOTrade.csv', index = False)
final

,Country,FIPS_CODE,1976,1977,1978,1979,1980,1981,1982,1983,...,2014,2015,2016,2017,2018,2019,2020,2021,Earliest,MostRecent
0,Afghanistan,AFG,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,NaN,NaN,...,0.0000,0.0000,0.0000,0.0450,0.0380,0.0271,0.0381,0.0389,0.0000,0.0389
1,Albania,ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.3302,0.4165,0.4549,0.5821,0.5756,0.5656,0.6182,0.6928,0.0000,0.6928
2,Algeria,DZA,0.0185,0.0357,0.0264,0.0255,0.0221,0.0307,0.0289,0.0401,...,0.0638,0.0662,0.0684,0.0728,0.0759,0.0769,0.0823,0.0640,0.0185,0.0640
3,Angola,AGO,NaN,NaN,NaN,NaN,0.0000,0.0000,1.0335,0.8155,...,0.2368,0.2609,0.2640,0.2946,0.2404,0.1268,0.0613,0.0416,0.0000,0.0416
4,Argentina,ARG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0070,...,0.0303,0.0296,0.0338,0.0344,0.0437,0.0368,0.0411,0.0450,0.0070,0.0450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,"Venezuela, Bolivarian Republic",VEN,0.0240,0.0309,0.0456,0.0444,0.0433,0.0442,0.0458,0.0148,...,0.0733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0240,0.0733
184,Viet Nam,VNM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.5525,0.5437,0.5315,0.6277,0.5996,0.5648,0.5618,0.5725,0.0000,0.5725
185,Yemen,YEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0698,0.0484,0.0623,0.0727,0.0627,NaN,NaN,NaN,0.0160,0.0627
186,Zambia,ZMB,0.0653,0.1032,0.2678,0.2182,0.0644,0.0607,0.0615,0.0535,...,0.2999,0.5497,0.4830,0.5006,0.4392,0.5062,0.5076,0.5985,0.0653,0.5985


# AquacultureProduction

In [4]:
df = pd.read_excel('DataDict_FAOFish.xlsx')
df = df[df['Notes'] != 'KS']
df

,Variable,Table,Group,SubGroup,Series,CoVaTrA,Cohort,Definition,Extended Source Defn,Units,...,Proprietary,Name in Source,UsedInPreprocessor,UsedInPreprocessorFileName,CompareOtherForecast,Code in Source,Decimal Places,Country Concordance,Formula,DisplayNotes
0,AgFishAquaProdAqAnimalsFSJ,SeriesAgFishAquaProdAqAnimalsFSJ,Agriculture,Fish,Yes,No,No,Total Aquatic Animals Aquaculture Production (...,NaN,Tonnes,...,0,NaN,1,AGRI,0,Aquatic Animals NEI,4,FAO Fishstat,NaN,NaN
1,AgFishAquaProdAqPlantsFSJ,SeriesAgFishAquaProdAqPlantsFSJ,Agriculture,Fish,Yes,No,No,Total Aquatic Plants Aquaculture Production (t...,NaN,Tonnes,...,0,Aquatic Plants global aquaculture production; ...,1,AGRI,0,Aquatic Plants,4,FAO Fishstat,NaN,NaN
2,AgFishAquaProdCephalopodsFSJ,SeriesAgFishAquaProdCephalopodsFSJ,Agriculture,Fish,Yes,No,No,Total Cephalopods Aquaculture Production (tonn...,NaN,Tonnes,...,0,Cephalopods global aquaculture production; All...,1,AGRI,0,Cephalopods,4,FAO Fishstat,NaN,NaN
3,AgFishAquaProdCrustaceansFSJ,SeriesAgFishAquaProdCrustaceansFSJ,Agriculture,Fish,Yes,No,No,Total Crustaceans Aquaculture Production (tonn...,NaN,Tonnes,...,0,Total Demersal Aquaculture Production (tonnes...,1,AGRI,0,Crustaceans,4,FAO Fishstat,NaN,NaN
4,AgFishAquaProdDemersalFSJ,SeriesAgFishAquaProdDemersalFSJ,Agriculture,Fish,Yes,No,No,Total Demersal Aquaculture Production (tonnes...,NaN,Tonnes,...,0,Total Demersal Aquaculture Production (tonnes...,1,AGRI,0,Demersal Marine Fish,4,FAO Fishstat,NaN,NaN
5,AgFishAquaProdFreshwaterFSJ,SeriesAgFishAquaProdFreshwaterFSJ,Agriculture,Fish,Yes,No,No,Total Freshwater Aquaculture Production (tonne...,NaN,Tonnes,...,0,Freshwater and Diadromous Fish aquaculture pro...,1,AGRI,0,Freshwater and Diadromous Fish,4,FAO Fishstat,NaN,NaN
6,AgFishAquaProdMarineFSJ,SeriesAgFishAquaProdMarineFSJ,Agriculture,Fish,Yes,No,No,Total Marine Aquaculture Production (tonnes) f...,NaN,Tonnes,...,0,Marine Fish NEI global aquaculture production;...,1,AGRI,0,Marine Fish NEI,4,FAO Fishstat,NaN,NaN
7,AgFishAquaProdMolluscsFSJ,SeriesAgFishAquaProdMolluscsFSJ,Agriculture,Fish,Yes,No,No,Total Molluscs Aquaculture Production (tonnes)...,NaN,Tonnes,...,0,Molluscs excl. cephalopods global aquaculture ...,1,AGRI,0,Molluscs excl. Cephalopods,4,FAO Fishstat,NaN,NaN
8,AgFishAquaProdPelagicFSJ,SeriesAgFishAquaProdPelagicFSJ,Agriculture,Fish,Yes,No,No,Total Pelagic Aquaculture Production (tonnes) ...,NaN,Tonnes,...,0,Pelagic marine fish global aquaculture product...,1,AGRI,0,Pelagic Marine Fish,4,FAO Fishstat,NaN,NaN
9,AgFishExportQuantityFAOTrade,SeriesAgFishExportQuantityFAOTrade,"Agriculture, Trade",Trade,Yes,No,No,Quantity of fish exported(Tonnes) from FishstatJ,Quantity of fish exported (Tonnes) from Fishst...,Tonnes,...,0,NaN,1,AGRI,0,NaN,4,FAO Fishstat,NaN,NaN


In [5]:
ap = df.head(9)
for i in range(len(ap)):
    v = ap.iloc[i,0]
    s = ap.iloc[i,24]
    print(v, s)

AgFishAquaProdAqAnimalsFSJ Aquatic Animals NEI
AgFishAquaProdAqPlantsFSJ Aquatic Plants
AgFishAquaProdCephalopodsFSJ Cephalopods
AgFishAquaProdCrustaceansFSJ Crustaceans
AgFishAquaProdDemersalFSJ Demersal Marine Fish
AgFishAquaProdFreshwaterFSJ Freshwater and Diadromous Fish
AgFishAquaProdMarineFSJ Marine Fish NEI
AgFishAquaProdMolluscsFSJ Molluscs excl. Cephalopods
AgFishAquaProdPelagicFSJ Pelagic Marine Fish


## Loop to generate the files

In [6]:
country = pd.read_excel('Country.xlsx', sheet_name='AquaculturePro')
country_new = pd.read_excel('IFsCountry.xlsx')

col = ['Country (Name)']
for y in range(1950,2022):
    col.append(f'[{y}]')

col_new = ['Country (Name)']
for y in range(1950,2022):
    col_new.append(y)

In [7]:
for i in range(len(ap)):
    v = ap.iloc[i,0]
    s = ap.iloc[i,24]
    print(v, s)
    dt = pd.read_csv('AquacultureProduction-1.csv', encoding='windows-1252')
    dt = dt[dt['ASFIS species (Name)'] == s]
    dt = dt[col]
    dt.columns = col_new
    dt = dt.set_index('Country (Name)')
    dt = dt.replace("...","")
    dt = pd.DataFrame(dt.stack(dropna = False), 
                    columns = ['val']).reset_index().rename(columns={'level_1':'year'})

    dt = country.merge(dt, left_on = 'FAOFISH_Country', right_on = 'Country (Name)', how = 'left')
    dt = dt.drop(columns=['FAOFISH_Country', 'Country (Name)'])
    dt = dt.groupby(['Country', 'FIPS_CODE', 'year']).sum().reset_index()

    dt.year = dt.year.astype("Int64")

    dt1 = dt[['Country','year','val']]
    first_last = dt1.sort_values(['Country','year']).groupby(dt1['Country'])['val'] \
            .agg(['first', 'last']).reset_index()
    first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})

    dt = dt.pivot_table(index = ['Country','FIPS_CODE'],
                        columns='year',
                        values = 'val').reset_index()

    dt = dt.merge(first_last,on=['Country'],how='left')
    dt = country_new.merge(dt, on = ['Country','FIPS_CODE'], how = 'left')
    print(v,dt.shape)

    # first_last

    # dt.to_csv(f'IFs Import/{v}.csv', index = False)

AgFishAquaProdAqAnimalsFSJ Aquatic Animals NEI
AgFishAquaProdAqAnimalsFSJ (188, 76)
AgFishAquaProdAqPlantsFSJ Aquatic Plants
AgFishAquaProdAqPlantsFSJ (188, 76)
AgFishAquaProdCephalopodsFSJ Cephalopods
AgFishAquaProdCephalopodsFSJ (188, 76)
AgFishAquaProdCrustaceansFSJ Crustaceans
AgFishAquaProdCrustaceansFSJ (188, 76)
AgFishAquaProdDemersalFSJ Demersal Marine Fish
AgFishAquaProdDemersalFSJ (188, 76)
AgFishAquaProdFreshwaterFSJ Freshwater and Diadromous Fish
AgFishAquaProdFreshwaterFSJ (188, 76)
AgFishAquaProdMarineFSJ Marine Fish NEI
AgFishAquaProdMarineFSJ (188, 76)
AgFishAquaProdMolluscsFSJ Molluscs excl. Cephalopods
AgFishAquaProdMolluscsFSJ (188, 76)
AgFishAquaProdPelagicFSJ Pelagic Marine Fish
AgFishAquaProdPelagicFSJ (188, 76)


# Export/Import

## AgFishExportQuantityFAOTrade

In [ ]:
country = pd.read_excel('Country.xlsx', sheet_name='ExportImport')
# country_new = pd.read_excel(r'C:\Users\panda\OneDrive - University of Denver\Pardee 2024 Winter\IFsCountry.xlsx')

col = ['Reporting country (Name)']
for y in range(1976,2022):
    col.append(f'[{y}]')

col_new = ['Reporting country (Name)']
for y in range(1976,2022):
    col_new.append(y)
    
dt = pd.read_csv('ExportImportQuantity-1.csv', encoding='windows-1252')
dt = dt[dt['Trade flow (Name)'] == 'Exports']
dt = dt[col]
dt.columns = col_new
dt = dt.set_index('Reporting country (Name)')
dt = dt.replace("...","")
dt = pd.DataFrame(dt.stack(dropna = False), 
                  columns = ['val']).reset_index().rename(columns={'level_1':'year'})

dt = country.merge(dt, left_on = 'FAOFISH_Country', right_on = 'Reporting country (Name)', how = 'left')
dt = dt.drop(columns=['FAOFISH_Country', 'Reporting country (Name)'])
dt = dt.groupby(['Country', 'FIPS_CODE', 'year']).sum().reset_index()

dt.year = dt.year.astype("Int64")

# dt.val = dt.val*1000
dt1 = dt[['Country','year','val']]
first_last = dt1.sort_values(['Country','year']).groupby(dt1['Country'])['val'] \
        .agg(['first', 'last']).reset_index()
first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})

dt = dt.pivot_table(index = ['Country','FIPS_CODE'],
                    columns='year',
                    values = 'val').reset_index()

dt = dt.merge(first_last,on=['Country'],how='left')
dt = country_new.merge(dt, on = ['Country','FIPS_CODE'], how = 'left')
dt

## AgFishImportQuantityFAOTrade

In [ ]:
country = pd.read_excel('Country.xlsx', sheet_name='ExportImport')
# country_new = pd.read_excel(r'C:\Users\panda\OneDrive - University of Denver\Pardee 2024 Winter\IFsCountry.xlsx')

col = ['Reporting country (Name)']
for y in range(1976,2022):
    col.append(f'[{y}]')

col_new = ['Reporting country (Name)']
for y in range(1976,2022):
    col_new.append(y)
    
dt = pd.read_csv('ExportImportQuantity-1.csv', encoding='windows-1252')
dt = dt[dt['Trade flow (Name)'] == 'Imports']
dt = dt[col]
dt.columns = col_new
dt = dt.set_index('Reporting country (Name)')
dt = dt.replace("...","")
dt = pd.DataFrame(dt.stack(dropna = False), 
                  columns = ['val']).reset_index().rename(columns={'level_1':'year'})

dt = country.merge(dt, left_on = 'FAOFISH_Country', right_on = 'Reporting country (Name)', how = 'left')
dt = dt.drop(columns=['FAOFISH_Country', 'Reporting country (Name)'])
dt = dt.groupby(['Country', 'FIPS_CODE', 'year']).sum().reset_index()

dt.year = dt.year.astype("Int64")

# dt.val = dt.val*1000
dt1 = dt[['Country','year','val']]
first_last = dt1.sort_values(['Country','year']).groupby(dt1['Country'])['val'] \
        .agg(['first', 'last']).reset_index()
first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})

dt = dt.pivot_table(index = ['Country','FIPS_CODE'],
                    columns='year',
                    values = 'val').reset_index()

dt = dt.merge(first_last,on=['Country'],how='left')
dt = country_new.merge(dt, on = ['Country','FIPS_CODE'], how = 'left')
dt